In [3]:
%pip install -qU langchain-pinecone pinecone-notebooks
!pip install -qU langchain_google_genai tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00


In [4]:
from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
userdata.get('API_KEY_PINECONE')

pinecone_api_key = userdata.get("API_KEY_PINECONE")

pc = Pinecone(api_key=pinecone_api_key)

In [5]:
import time

index_name = "project-02-rag"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
from uuid import uuid4

from langchain_core.documents import Document

my_document = Document("/content/Agentic AI_ 4 reasons why it’s the next big thing in AI research.html")

vector_store.add_documents([my_document])

['cb5925bc-e1c4-41ef-bec1-03850680c462']

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open("/content/Agentic AI_ 4 reasons why it’s the next big thing in AI research.html", "r") as f:
    my_document_content = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
my_document = Document(page_content=my_document_content)
documents = text_splitter.split_documents([my_document])
vector_store.add_documents(documents)

['8e04de68-498a-4ea3-87ea-4c0b05bda099',
 '494b43ee-47c0-456e-9deb-7dbd90862fae',
 'c92caf31-0a34-44bc-bd28-9669c867f2b3',
 '66d19f64-38e7-4f10-bb23-52e9b82febe7',
 '6e74bb84-6219-437e-b086-cd35acc691eb',
 '78a79551-5a66-4496-82b3-18e2ea37e99f',
 'ab1071e6-923f-4c28-a9b9-ce8b02399078',
 'fa8b7a7e-6db4-4767-a543-0a4e8399242c',
 'c78f9600-7d44-4026-9af3-c726324c3877',
 '4c1d22db-aacc-4f68-8149-0cbc7544a1ad',
 'dabd42d9-7a71-4e11-8497-4515bdead6f0',
 '94b2072a-4981-4480-b652-8fda8efbe4f6',
 '50c28fe9-ffaf-477f-a357-46b28fc26a35',
 '764debb0-52fe-4248-b546-e4f9849fc65b',
 'c56e198e-3822-4b80-8444-99da47ba1573',
 'bd6d5149-8b73-44de-a9dc-ed5d1dcd8321',
 '9155751f-9c3b-41a8-a3fb-db30d02e0813',
 '4548a2ed-252e-4030-9dca-6a64110f6eac',
 'd04f87d9-3faf-4a03-a5f5-66d3f76bba37',
 '255c34ce-3203-4690-8f16-8337f640db94',
 'd4ee1222-c5dc-4305-9527-136717b356b0',
 'a0f60a8a-a95f-45d7-b1ba-068fb41fd110',
 '75532e6b-6bd8-4f28-a01b-41ad6c0859c3',
 'e2238990-c5df-41ae-8a03-4e4d538c1392',
 '4c6f063d-eaca-

In [11]:
for i, doc in enumerate(documents):
    doc.metadata = {"source": f"source_{i}"}

In [12]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.5},
)

In [13]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(
    api_key= "GOOGLE_API_KEY",
    model="gemini-2.0-flash-exp",
    temperature= 1
)

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="stuff",
)

In [18]:
query = "What is the history of artificial intelligence?"
response = qa_chain.run(query)
response

"I don't know the answer.\n"